# Battle of the neighbourhoods - Toronto

**Mohamed Amine GHOUWATI**

This notebook contains all 3 questions for week 3 assignment

## Question 1

In [1]:
#importing the required libraries

import numpy as np 

import pandas as pd 
import json 

import requests 
!pip install geopy
from geopy.geocoders import Nominatim 

from IPython.display import Image 
from IPython.core.display import HTML 
    

from pandas.io.json import json_normalize

! pip install folium==0.5.0
import folium 


import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt 
%matplotlib inline 

from sklearn.cluster import KMeans

     |████████████████████████████████| 112 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 1.9 MB/s eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=bef15531b103a7ad4610fdf15728a8087ed741d6f94c9b355bab98109a200d22
  Stored in directory: /Users/maghouwa/Library/Caches/pip/wheels/ef/4c/4a/17fd3d7fb7b6243d5a7a8d165870cd5c6ad2ec4c0582f039e4
Successfully built folium
Folium installed


In [2]:
from bs4 import BeautifulSoup
import requests 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib") 

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [4]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [8]:
df.shape, df.Neighborhood.str.count("Not assigned").sum()

((103, 3), 0)

We can see that we have 103 rows and 3 columns with no "Not assigned" Neighbourhoods

## Question 2

In [9]:
data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [10]:
df.dtypes, data.dtypes

(PostalCode      object
 Borough         object
 Neighborhood    object
 dtype: object,
 Postal Code     object
 Latitude       float64
 Longitude      float64
 dtype: object)

Our dataframes are of same size and have the same type for 'PostalCode' which means we can join them

In [11]:
combined_data = df.join(data.set_index('Postal Code'), on='PostalCode', how='inner')
combined_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(combined_data['Borough'].unique()),
        combined_data.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


In [14]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Toronto are {}, {}.'.format(latitude, longitude))

Coordinates of Toronto are 43.6534817, -79.3839347.


In [15]:
neighborhoods=combined_data
# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In order to see the map you have to make this Notebook Trusted : File --> Trust Notebook

## Question 3

In [16]:
CLIENT_ID = 'ERVIEHR4XSEAHEPZGYAMLEU2KDRVWA1JRSYQ1CMKT2DRSX2Q' # your Foursquare ID
CLIENT_SECRET = 'PHDOY5Z3D4WZXSDOR4UWXU0TWQPA2MBFVGATUCW42D14S1PU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ERVIEHR4XSEAHEPZGYAMLEU2KDRVWA1JRSYQ1CMKT2DRSX2Q
CLIENT_SECRET:PHDOY5Z3D4WZXSDOR4UWXU0TWQPA2MBFVGATUCW42D14S1PU


In [17]:
combined_data.rename(columns = {'Neighborhood':'Neighbourhood'}, inplace = True)
# Creating the map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for latitude, longitude, borough, neighbourhood in zip(combined_data['Latitude'], combined_data['Longitude'], combined_data['Borough'], combined_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_Toronto)  
    
map_Toronto

In order to see the map you have to make this Notebook Trusted : File --> Trust Notebook

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

Venues for each neibourhood

In [19]:
venues_in_toronto = getNearbyVenues(combined_data['Neighbourhood'], combined_data['Latitude'], combined_data['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [20]:
venues_in_toronto.shape

(1325, 5)

In [21]:
venues_in_toronto.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Parkwoods,43.753259,-79.329656,KFC,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,Park
2,Parkwoods,43.753259,-79.329656,Variety Store,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,Portuguese Restaurant


In [22]:
venues_in_toronto.groupby('Neighbourhood').head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Parkwoods,43.753259,-79.329656,KFC,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,Park
2,Parkwoods,43.753259,-79.329656,Variety Store,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,Portuguese Restaurant
...,...,...,...,...,...
1311,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,South St. Burger,Burger Joint
1312,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Dollarama,Discount Store
1313,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Healthy Planet,Supplement Shop
1314,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Wingporium,Wings Joint


In [23]:
venues_in_toronto.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Accessories Store,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Ardene Shoes Outlet
Adult Boutique,Church and Wellesley,43.665860,-79.383160,Seduction
Airport,Downsview East,43.737473,-79.394420,Toronto Downsview Airport (YZD)
Airport Food Court,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Billy Bishop Café
Airport Gate,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Gate 8
...,...,...,...,...
Warehouse Store,Thorncliffe Park,43.705369,-79.349372,Costco
Wine Bar,"Toronto Dominion Centre, Design Exchange",43.653206,-79.379817,The National Club
Wings Joint,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Wingporium


In [24]:
toronto_venue_cat = pd.get_dummies(venues_in_toronto[['Venue Category']], prefix="", prefix_sep="")
toronto_venue_cat

,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1321,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1322,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_venue_cat['Neighbourhood'] = venues_in_toronto['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [toronto_venue_cat.columns[-1]] + list(toronto_venue_cat.columns[:-1])
toronto_venue_cat = toronto_venue_cat[fixed_columns]

toronto_venue_cat.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
toronto_grouped = toronto_venue_cat.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038462,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Clothing Store,Breakfast Spot,Dim Sum Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Coffee Shop,Pub,Sandwich Place,Skating Rink,Gym,Airport Service,Department Store,Escape Room
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Mobile Phone Shop,Ice Cream Shop,Diner,Bridal Shop,Sandwich Place,Restaurant,Deli / Bodega,Supermarket
3,Bayview Village,Bank,Chinese Restaurant,Café,Japanese Restaurant,Yoga Studio,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Comfort Food Restaurant,Thai Restaurant,Restaurant,Juice Bar,Japanese Restaurant,Fast Food Restaurant,Indian Restaurant


### Now we are going to design the model to cluster our Neighbourhoods

In [29]:
# set number of clusters
k_num_clusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k_num_clusters, random_state=0).fit(toronto_grouped_clustering)
kmeans

KMeans(n_clusters=5, random_state=0)

In [31]:
kmeans.labels_[0:100]

array([2, 2, 2, 0, 0, 0, 0, 0, 2, 4, 2, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0,
       0, 2, 2, 4, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2,
       3, 2, 2, 0, 0, 2, 2, 4, 0, 0, 2, 4, 2, 2, 0, 2, 0, 0, 3, 0, 0, 2,
       4, 0, 0, 4, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 1, 0, 4, 2, 0,
       0, 2, 0, 2, 1, 2, 0, 2, 4, 0, 2, 1], dtype=int32)

In [32]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [33]:
toronto_merged = combined_data

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Fast Food Restaurant,Park,Food & Drink Shop,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Coffee Shop,Eastern European Restaurant,Drugstore,Donut Shop,Deli / Bodega,Distribution Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Event Space,Beer Store,Restaurant,Café,Theater
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Miscellaneous Shop,Boutique,Event Space,Vietnamese Restaurant,General Entertainment,Eastern European Restaurant
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Fried Chicken Joint,Sandwich Place,Park,Nightclub,Mexican Restaurant,Japanese Restaurant,Italian Restaurant


In [34]:
toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])

#### Now we are going to plot the clusters in a map

In [35]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['Latitude'], toronto_merged_nonan['Longitude'], toronto_merged_nonan['Neighbourhood'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters)
        
map_clusters

In order to see the map you have to make this Notebook Trusted : File --> Trust Notebook

Now we are going to check each of our five clusters

In [36]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 0, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0.0,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Event Space,Beer Store,Restaurant,Café,Theater
4,Queen's Park,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Fried Chicken Joint,Sandwich Place,Park,Nightclub,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
7,North York,0.0,Gym,Caribbean Restaurant,Café,Japanese Restaurant,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
9,Downtown Toronto,0.0,Café,Clothing Store,Japanese Restaurant,Middle Eastern Restaurant,Theater,Plaza,Sandwich Place,Burger Joint,Burrito Place,Steakhouse
10,North York,0.0,Park,Bakery,Japanese Restaurant,Italian Restaurant,Yoga Studio,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
11,Etobicoke,0.0,Filipino Restaurant,Bakery,Middle Eastern Restaurant,Yoga Studio,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
13,North York,0.0,Gym,Restaurant,Coffee Shop,Italian Restaurant,Dim Sum Restaurant,Bus Line,Supermarket,Asian Restaurant,Sandwich Place,Art Gallery
15,Downtown Toronto,0.0,Gastropub,Café,Farmers Market,Japanese Restaurant,Coffee Shop,Middle Eastern Restaurant,Italian Restaurant,Beer Bar,Diner,Restaurant
17,Etobicoke,0.0,Pizza Place,Beer Store,Pharmacy,Coffee Shop,Café,Shopping Plaza,Park,Liquor Store,Pet Store,Cupcake Shop
20,Downtown Toronto,0.0,Bakery,Seafood Restaurant,Farmers Market,Cocktail Bar,Beer Bar,Japanese Restaurant,Basketball Stadium,Sporting Goods Shop,Jazz Club,Bistro


In [37]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 1, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East York/East Toronto,1.0,Park,Convenience Store,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
64,York,1.0,Convenience Store,Yoga Studio,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
66,North York,1.0,Park,Convenience Store,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


In [38]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 2, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2.0,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Coffee Shop,Eastern European Restaurant,Drugstore,Donut Shop,Deli / Bodega,Distribution Center
3,North York,2.0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Miscellaneous Shop,Boutique,Event Space,Vietnamese Restaurant,General Entertainment,Eastern European Restaurant
6,Scarborough,2.0,Fast Food Restaurant,Print Shop,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
8,East York,2.0,Pizza Place,Athletics & Sports,Café,Pet Store,Bank,Flea Market,Intersection,Pharmacy,Gym / Fitness Center,Gastropub
12,Scarborough,2.0,Construction & Landscaping,Bar,Home Service,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore
14,East York,2.0,Beer Store,Skating Rink,Dance Studio,Curling Ice,Video Store,Athletics & Sports,Park,Electronics Store,Eastern European Restaurant,Drugstore
16,York,2.0,Field,Hockey Arena,Trail,Tennis Court,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
18,Scarborough,2.0,Restaurant,Medical Center,Breakfast Spot,Donut Shop,Mexican Restaurant,Bank,Intersection,Rental Car Location,Electronics Store,Dessert Shop
19,East Toronto,2.0,Neighborhood,Asian Restaurant,Trail,Health Food Store,Pub,Discount Store,Dessert Shop,Dim Sum Restaurant,Diner,Yoga Studio
26,Scarborough,2.0,Thai Restaurant,Gas Station,Bank,Bakery,Athletics & Sports,Fried Chicken Joint,Caribbean Restaurant,Lounge,Hakka Restaurant,Creperie


In [39]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 3, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,3.0,Baseball Field,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
101,Etobicoke,3.0,Baseball Field,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


In [40]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 4, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4.0,Fast Food Restaurant,Park,Food & Drink Shop,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
21,York,4.0,Park,Women's Store,Pool,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
40,North York,4.0,Airport,Park,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
52,North York,4.0,Park,Yoga Studio,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
61,Central Toronto,4.0,Bus Line,Park,Swim School,Yoga Studio,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
85,Scarborough,4.0,Playground,Park,Intersection,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
91,Downtown Toronto,4.0,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
98,Etobicoke,4.0,River,Park,Pool,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


Thank you for reviewing my assignment